In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("FinalBalancedDataset.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43488 entries, 0 to 43487
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  43488 non-null  int64 
 1   Toxicity    43488 non-null  int64 
 2   tweet       43488 non-null  object
dtypes: int64(2), object(1)
memory usage: 1019.4+ KB


In [4]:
data.head(5)

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [5]:
data = data.drop("Unnamed: 0", axis=1)

In [6]:
data.head(5)

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [7]:
data['Toxicity'].value_counts()

Toxicity
0    31038
1    12450
Name: count, dtype: int64

## Text pre-processing

In [8]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
wordnet_lemmatizer = WordNetLemmatizer()

In [10]:
import re

In [11]:
def prepare_text(text):
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = text.split()
    text = ' '.join(text)
    text = word_tokenize(text)
    text = pos_tag(text)
    lemma = []
    for i in text: lemma.append(wordnet_lemmatizer.lemmatize(i[0], pos = get_wordnet_pos(i[1])))
    lemma = ' '.join(lemma)
    return lemma

In [12]:
data['clean_tweets'] = data['tweet'].apply(lambda x: prepare_text(x))

In [13]:
data.head(5)

,Toxicity,tweet,clean_tweets
0,0,@user when a father is dysfunctional and is s...,user when a father be dysfunctional and be so ...
1,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i ca n't use ...
2,0,bihday your majesty,bihday your majesty
3,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,0,factsguide: society now #motivation,factsguide society now motivation


## Tf-idf for features

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [15]:
corpus = data['clean_tweets'].values.astype('U')

In [18]:
stopwords = set(nltk_stopwords.words('english'))

In [22]:
count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))
tf_idf = count_tf_idf.fit_transform(corpus)

In [23]:
import pickle

In [24]:
pickle.dump(count_tf_idf, open("tf_idf.pkt", "wb"))

In [26]:
tf_idf_train, tf_idf_test, target_train, target_test = train_test_split(
    tf_idf, data['Toxicity'], test_size = 0.7, random_state= 42, shuffle=True
)

## Create a Binary Classification Model

In [27]:
model_bayes = MultinomialNB()

In [28]:
model_bayes = model_bayes.fit(tf_idf_train, target_train)

In [29]:
y_pred_proba = model_bayes.predict_proba(tf_idf_test)[::, 1]

In [30]:
y_pred_proba

array([0.67568876, 0.12669515, 0.46028184, ..., 0.07518092, 0.03057116,
       0.18859304])

In [31]:
fpr, tpr, _ = roc_curve(target_test, y_pred_proba)

In [32]:
final_roc_auc = roc_auc_score(target_test, y_pred_proba)

In [33]:
final_roc_auc

0.9558717244625633

In [41]:
test_text = "You are Ugly!"
test_tfidf = count_tf_idf.transform([test_text])
display(model_bayes.predict_proba(test_tfidf))
display(model_bayes.predict(test_tfidf))

array([[0.36689414, 0.63310586]])

array([1])

## Save the model

In [39]:
pickle.dump(model_bayes, open("toxicity_model.pkt", "wb"))